<img src="https://blog.sodio.tech/wp-content/uploads/2018/10/Geolocation.gif">

# Nearby locations

Now we have to find the best `location` for the company to meet the needs of the employees.

These requirements are: 

* 30% of the company staff have at least 1 child.
* Executives like `Starbucks` A LOT.
* Account managers need to `travel` a lot.
* Everyone in the company is between 25 and 40, give them some `place to go party`.
* The CEO is `vegan`.
* If you want to make the maintenance guy happy, a `basketball` stadium must be around 10 Km.
* The office dog—"Dobby" needs a `hairdresser` every month. Ensure there's one not too far away.


<img src="https://www.gif-vif.com/Gaming-with-doggo.gif">

## Import the necessary libraries

In [2]:
from pymongo import MongoClient
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import requests
import json

import src.limpieza_texto as lt
import src.visualizacion as vt

from pandas import json_normalize
from functools import reduce
import operator

from dotenv import load_dotenv
import os
load_dotenv()

True

## Conect with `FourSquare`

In [13]:
tok1 = os.getenv("tok_id")
tok2 = os.getenv("tok_s")

In [24]:
san_francisco = {'type': 'Point', 'coordinates': [37.7579, -122.4192]}

In [66]:
vegan = "4bf58dd8d48988d1d3941735"

In [65]:
f"{vegan}"

'hola'

In [26]:
parametros = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{san_francisco.get('coordinates')[0]},{san_francisco.get('coordinates')[1]}",
    "query": f"{vegan}", 
    "limit": 50    
}

In [27]:
resp = requests.get(url= url_query, params = parametros).json()

In [44]:
#resp

In [29]:
resp.keys()

dict_keys(['meta', 'response'])

In [30]:
data = resp.get("response")

In [32]:
#data

In [61]:
decode = resp.get("response").get("groups")[0]

In [62]:
from pandas import json_normalize
json_normalize(decode)

,type,name,items
0,Recommended Places,recommended,"[{'reasons': {'count': 0, 'items': [{'summary'..."


In [35]:
decode_otravez = decode.get("items")

In [36]:
from functools import reduce
import operator

In [37]:
mapa_nombre =  ["venue", "name"]
mapa_latitud = ["venue", "location", "lat"]
mapa_longitud = ["venue", "location", "lng"]


In [38]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [55]:
coord = []
for dic in decode_otravez:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    coord.append(paralista)

In [56]:
dataframe = pd.DataFrame(coord)
dataframe.head()

,name,latitud,longitud
0,Bowl'd Acai,37.761549,-122.421400
1,Louise M. Davies Symphony Hall,37.777976,-122.420157
2,K&D Market,37.764834,-122.423667
3,H.D. Buttercup,37.765845,-122.400239
4,Tazza D'Amore,37.764198,-122.429977


In [57]:
documentos = []
for diccionario in coord:
    temporal = {
        "name" : diccionario.get("name"),
        "location" : {"type" : "Point", "coordinates" : [diccionario.get("latitud"), diccionario.get("longitud")]}
    }
    documentos.append(temporal)

In [58]:
documentos

[{'name': "Bowl'd Acai",
  'location': {'type': 'Point',
   'coordinates': [37.761549481103955, -122.42140002967778]}},
 {'name': 'Louise M. Davies Symphony Hall',
  'location': {'type': 'Point',
   'coordinates': [37.7779761642689, -122.42015694896058]}},
 {'name': 'K&D Market',
  'location': {'type': 'Point',
   'coordinates': [37.764834498423426, -122.42366728791336]}},
 {'name': 'H.D. Buttercup',
  'location': {'type': 'Point', 'coordinates': [37.765845, -122.400239]}},
 {'name': "Tazza D'Amore",
  'location': {'type': 'Point',
   'coordinates': [37.764198303222656, -122.42997741699219]}}]

In [59]:
df = pd.DataFrame(documentos)

In [60]:
df.to_json("./json/restaurantes_veganos.json", orient = "records")